<a href="https://colab.research.google.com/github/tul17ii/Mini-Fitness-Function-Project/blob/main/cost_with_best_strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import math

# Define farming-related options
crop_types = ["Wheat", "Corn", "Rice", "Soybean"]
planting_times = ["Spring", "Summer", "Fall", "Winter"]
fertilizers = ["Nitrogen", "Phosphorus", "Potassium", "Organic"]
pesticides = ["Herbicide", "Insecticide", "Fungicide", "None"]
soil_nutrients = ["Low", "Medium", "High"]
farm_sizes = ["Small", "Medium", "Large"]
farm_types = ["Organic", "Conventional"]

# Simulated database for fertilizers and pesticides
fertilizers_db = {
    "Nitrogen": {"unit_cost": 30, "efficiency": 0.8, "env_impact": 15},
    "Phosphorus": {"unit_cost": 25, "efficiency": 0.7, "env_impact": 12},
    "Potassium": {"unit_cost": 20, "efficiency": 0.6, "env_impact": 10},
    "Organic": {"unit_cost": 15, "efficiency": 0.5, "env_impact": 5},
}

pesticides_db = {
    "Herbicide": {"unit_cost": 20, "efficiency": 0.7, "env_impact": 10},
    "Insecticide": {"unit_cost": 25, "efficiency": 0.8, "env_impact": 15},
    "Fungicide": {"unit_cost": 30, "efficiency": 0.9, "env_impact": 20},
    "None": {"unit_cost": 0, "efficiency": 0, "env_impact": 0},
}

# Normalize input to title case
def normalize_input(user_input):
    return [item.strip().title() for item in user_input.split(',')]

# Fitness functions
def crop_yield_fitness(chromosome):
    return random.uniform(50, 100)  # Simulated crop yield score

def environmental_impact_fitness(chromosome):
    return random.uniform(10, 50)  # Simulated environmental impact score

def combined_fitness(chromosome):
    fertilizer = chromosome[2]
    pesticide = chromosome[3]
    soil_requirement = chromosome[4]

    # Use simulated values
    efficiency = (
        fertilizers_db[fertilizer]["efficiency"] +
        pesticides_db[pesticide]["efficiency"]
    ) / 2

    env_impact = (
        fertilizers_db[fertilizer]["env_impact"] +
        pesticides_db[pesticide]["env_impact"]
    )
    return efficiency * 100 - env_impact

# Generate random chromosome
def generate_random_chromosome():
    return [
        random.choice(crop_types),
        random.choice(planting_times),
        random.choice(fertilizers),
        random.choice(pesticides),
        random.choice(soil_nutrients),
        random.choice(farm_sizes),
        random.choice(farm_types),
    ]

# Mutation (perturb the solution slightly)
def mutate(chromosome):
    mutation_point = random.randint(0, len(chromosome) - 1)
    mutation_options = [
        crop_types, planting_times, fertilizers, pesticides,
        soil_nutrients, farm_sizes, farm_types
    ]
    chromosome[mutation_point] = random.choice(mutation_options[mutation_point])
    return chromosome

# Simulated Annealing Algorithm
def simulated_annealing(fitness_function, initial_temp=100, cooling_rate=0.95, max_iterations=1000):
    # Step 1: Initialize a random solution
    current_solution = generate_random_chromosome()
    current_fitness = fitness_function(current_solution)
    best_solution = current_solution
    best_fitness = current_fitness
    temperature = initial_temp

    for iteration in range(max_iterations):
        # Step 2: Generate a neighbor solution
        new_solution = mutate(current_solution.copy())
        new_fitness = fitness_function(new_solution)

        # Step 3: Decide whether to accept the new solution
        if new_fitness > current_fitness:
            # Accept if the new solution is better
            current_solution = new_solution
            current_fitness = new_fitness
        else:
            # Accept with a probability if the new solution is worse
            acceptance_probability = math.exp((new_fitness - current_fitness) / temperature)
            if random.random() < acceptance_probability:
                current_solution = new_solution
                current_fitness = new_fitness

        # Step 4: Update the best solution found
        if current_fitness > best_fitness:
            best_solution = current_solution
            best_fitness = current_fitness

        # Step 5: Cool down the temperature
        temperature *= cooling_rate
        if temperature < 1e-3:  # Stop if temperature is too low
            break

    return best_solution, best_fitness

# Process user inputs
def process_query():
    print("\nAnswer the following questions to help us optimize your farming strategy:")

    user_data = {
        'preferred_crops': input("1. What crops do you want to grow? (e.g., Wheat, Corn, Rice, Soybean): "),
        'current_fertilizers': input("2. What fertilizers do you have access to? (e.g., Nitrogen, Organic): "),
        'current_pesticides': input("3. What pesticides do you have access to? (e.g., Herbicide, None): "),
        'soil_condition': input("4. What is your soil condition? (Low, Medium, High nutrients): "),
        'farm_size': input("5. What is the size of your farm? (Small, Medium, Large): "),
        'farm_type': input("6. What type of farm do you operate? (Organic, Conventional): "),
        'budget': float(input("7. What is your total budget for farming (e.g., $500, $1000)? ")),
        'priority': input("8. What is your priority? (Yield, Environmental Impact, Balanced): ")
    }

    # Normalize inputs for fertilizers and pesticides
    user_data['current_fertilizers'] = normalize_input(user_data['current_fertilizers'])
    user_data['current_pesticides'] = normalize_input(user_data['current_pesticides'])

    print("\nChoose a fitness function based on your priority:")
    print("1. Maximize crop yield (focus on production)")
    print("2. Minimize environmental impact (focus on sustainability)")
    print("3. Balanced optimization (consider both yield and impact)")
    choice = input("Enter 1, 2, or 3: ")

    if choice == "1":
        fitness_function = crop_yield_fitness
    elif choice == "2":
        fitness_function = environmental_impact_fitness
    elif choice == "3":
        fitness_function = combined_fitness
    else:
        print("Invalid choice. Defaulting to balanced optimization.")
        fitness_function = combined_fitness

    return fitness_function, user_data

# Suggest the best fertilizers and pesticides within budget
def suggest_best_inputs(budget, user_data):
    # Get the fertilizers and pesticides user selected
    available_fertilizers = user_data['current_fertilizers']
    available_pesticides = user_data['current_pesticides']

    # Find all possible combinations within the budget
    best_combination = None
    best_env_fitness = -math.inf
    best_total_cost = float('inf')

    for fertilizer in available_fertilizers:
        for pesticide in available_pesticides:
            cost = fertilizers_db[fertilizer]["unit_cost"] + pesticides_db[pesticide]["unit_cost"]
            if cost <= budget:
                env_fitness = (
                    fertilizers_db[fertilizer]["efficiency"] +
                    pesticides_db[pesticide]["efficiency"]
                ) / 2 - (
                    fertilizers_db[fertilizer]["env_impact"] +
                    pesticides_db[pesticide]["env_impact"]
                )
                if env_fitness > best_env_fitness:
                    best_combination = (fertilizer, pesticide)
                    best_env_fitness = env_fitness
                    best_total_cost = cost

    return best_combination, best_total_cost

# Main execution
print("Welcome to the Intelligent Smart Farm Optimization System!")
fitness_function, user_data = process_query()
best_solution, best_fitness = simulated_annealing(fitness_function)

print(f"\nBest farming strategy based on your inputs: {best_solution}")
print(f"Fitness score of the solution: {best_fitness}")

# Suggest the best fertilizers and pesticides based on budget
best_combination, best_cost = suggest_best_inputs(user_data['budget'], user_data)
if best_combination:
    print(f"\nThe best combination of fertilizers and pesticides within your budget:")
    print(f"Fertilizer: {best_combination[0]}, Pesticide: {best_combination[1]}")
    print(f"Total cost: ${best_cost}")
else:
    print("\nNo suitable combination of fertilizers and pesticides found within your budget.")


Welcome to the Intelligent Smart Farm Optimization System!

Answer the following questions to help us optimize your farming strategy:
1. What crops do you want to grow? (e.g., Wheat, Corn, Rice, Soybean): rice
2. What fertilizers do you have access to? (e.g., Nitrogen, Organic): nitrogen
3. What pesticides do you have access to? (e.g., Herbicide, None): none
4. What is your soil condition? (Low, Medium, High nutrients): high
5. What is the size of your farm? (Small, Medium, Large): large
6. What type of farm do you operate? (Organic, Conventional): organic
7. What is your total budget for farming (e.g., $500, $1000)? 500
8. What is your priority? (Yield, Environmental Impact, Balanced): yield

Choose a fitness function based on your priority:
1. Maximize crop yield (focus on production)
2. Minimize environmental impact (focus on sustainability)
3. Balanced optimization (consider both yield and impact)
Enter 1, 2, or 3: 3

Best farming strategy based on your inputs: ['Wheat', 'Summer', 

In [ ]:
import random
import math

# Simulated database for fertilizers and pesticides
fertilizers_db = {
    "Nitrogen": {"unit_cost": 30, "efficiency": 0.8, "env_impact": 15},
    "Phosphorus": {"unit_cost": 25, "efficiency": 0.7, "env_impact": 12},
    "Potassium": {"unit_cost": 20, "efficiency": 0.6, "env_impact": 10},
    "Organic": {"unit_cost": 15, "efficiency": 0.5, "env_impact": 5},
}

pesticides_db = {
    "Herbicide": {"unit_cost": 20, "efficiency": 0.7, "env_impact": 10},
    "Insecticide": {"unit_cost": 25, "efficiency": 0.8, "env_impact": 15},
    "Fungicide": {"unit_cost": 30, "efficiency": 0.9, "env_impact": 20},
    "None": {"unit_cost": 0, "efficiency": 0, "env_impact": 0},
}

# Fitness function for combined optimization
def combined_fitness(chromosome, budget):
    fertilizer = chromosome["fertilizer"]
    pesticide = chromosome["pesticide"]

    # Cost calculation
    fertilizer_cost = fertilizers_db[fertilizer]["unit_cost"]
    pesticide_cost = pesticides_db[pesticide]["unit_cost"]
    total_cost = fertilizer_cost + pesticide_cost

    # Environmental impact and efficiency
    efficiency = (
        fertilizers_db[fertilizer]["efficiency"] +
        pesticides_db[pesticide]["efficiency"]
    ) / 2

    env_impact = (
        fertilizers_db[fertilizer]["env_impact"] +
        pesticides_db[pesticide]["env_impact"]
    )

    # Penalize solutions that exceed budget
    if total_cost > budget:
        return efficiency * 100 - env_impact - (total_cost - budget) * 0.5
    return efficiency * 100 - env_impact

# Generate random chromosome
def generate_random_chromosome():
    return {
        "fertilizer": random.choice(list(fertilizers_db.keys())),
        "pesticide": random.choice(list(pesticides_db.keys()))
    }

# Mutation (perturb the solution slightly)
def mutate(chromosome):
    if random.choice([True, False]):
        chromosome["fertilizer"] = random.choice(list(fertilizers_db.keys()))
    else:
        chromosome["pesticide"] = random.choice(list(pesticides_db.keys()))
    return chromosome

# Simulated Annealing Algorithm
def simulated_annealing(fitness_function, budget, initial_temp=100, cooling_rate=0.95, max_iterations=1000):
    current_solution = generate_random_chromosome()
    current_fitness = fitness_function(current_solution, budget)
    best_solution = current_solution
    best_fitness = current_fitness
    temperature = initial_temp

    for iteration in range(max_iterations):
        new_solution = mutate(current_solution.copy())
        new_fitness = fitness_function(new_solution, budget)

        if new_fitness > current_fitness:
            current_solution = new_solution
            current_fitness = new_fitness
        else:
            acceptance_probability = math.exp((new_fitness - current_fitness) / temperature)
            if random.random() < acceptance_probability:
                current_solution = new_solution
                current_fitness = new_fitness

        if current_fitness > best_fitness:
            best_solution = current_solution
            best_fitness = current_fitness

        temperature *= cooling_rate
        if temperature < 1e-3:
            break

    return best_solution, best_fitness

# Process user inputs
def process_query():
    print("\nAnswer the following questions to optimize your farming strategy:")
    user_data = {
        'preferred_crops': input("1. What crops do you want to grow? (e.g., Wheat, Corn, Rice, Soybean): "),
        'soil_condition': input("2. What is your soil condition? (Low, Medium, High nutrients): "),
        'farm_size': input("3. What is the size of your farm? (Small, Medium, Large): "),
        'farm_type': input("4. What type of farm do you operate? (Organic, Conventional): "),
        'budget': float(input("5. What is your total budget for farming (e.g., $500, $1000)? ")),
        'priority': input("6. What is your priority? (Yield, Environmental Impact, Balanced): ")
    }

    print("\nChoose a fitness function based on your priority:")
    print("1. Maximize crop yield (focus on production)")
    print("2. Minimize environmental impact (focus on sustainability)")
    print("3. Balanced optimization (consider both yield and impact)")
    choice = input("Enter 1, 2, or 3: ")

    if choice == "1":
        fitness_function = crop_yield_fitness
    elif choice == "2":
        fitness_function = environmental_impact_fitness
    elif choice == "3":
        fitness_function = combined_fitness
    else:
        print("Invalid choice. Defaulting to balanced optimization.")
        fitness_function = combined_fitness

    return fitness_function, user_data

# Main execution
print("Welcome to the Intelligent Smart Farm Optimization System!")
fitness_function, user_data = process_query()
best_solution, best_fitness = simulated_annealing(fitness_function, user_data['budget'])
fertilizer_cost = fertilizers_db[best_solution["fertilizer"]]["unit_cost"]
pesticide_cost = pesticides_db[best_solution["pesticide"]]["unit_cost"]
total_cost = fertilizer_cost + pesticide_cost

print(f"\nBest farming strategy based on your inputs:")
print(f"Fertilizer: {best_solution['fertilizer']} - Cost: ${fertilizer_cost}")
print(f"Pesticide: {best_solution['pesticide']} - Cost: ${pesticide_cost}")
print(f"Total Cost: ${total_cost}")
print(f"Remaining Budget: ${user_data['budget'] - total_cost:.2f}")

# Fitness details and user-friendly outputs
if total_cost > user_data['budget']:
    print(f"Warning: The total cost exceeds your budget by ${total_cost - user_data['budget']:.2f}.")
else:
    print(f"Success: The total cost is within your budget!")

print(f"\nFitness score of the selected solution: {best_fitness:.2f}")

Welcome to the Intelligent Smart Farm Optimization System!

Answer the following questions to optimize your farming strategy:
1. What crops do you want to grow? (e.g., Wheat, Corn, Rice, Soybean): corn
2. What is your soil condition? (Low, Medium, High nutrients): high
3. What is the size of your farm? (Small, Medium, Large): large
4. What type of farm do you operate? (Organic, Conventional): organic
5. What is your total budget for farming (e.g., $500, $1000)? 1000
6. What is your priority? (Yield, Environmental Impact, Balanced): balanced

Choose a fitness function based on your priority:
1. Maximize crop yield (focus on production)
2. Minimize environmental impact (focus on sustainability)
3. Balanced optimization (consider both yield and impact)
Enter 1, 2, or 3: 3

Best farming strategy based on your inputs:
Fertilizer: Nitrogen - Cost: $30
Pesticide: Fungicide - Cost: $30
Total Cost: $60
Remaining Budget: $940.00
Success: The total cost is within your budget!

Fitness score of th